In [ ]:
def download_folder_s3(prefix_,download_to_path):
    import utils as u
    import os

    if not 'workbookDir' in globals():
        workbookDir = os.getcwd()

    aws_separator = "/"

    # Import S3 connectors librairies
    #from awsPy.aws_authorization import aws_connector
    import importlib.util
    spec = importlib.util.spec_from_file_location("aws_connector.py", "C:/Users/Hp/Documents/GitHub/aws-python/awsPy/aws_authorization/aws_connector.py")
    aws_connector = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(aws_connector)

    #from awsPy.aws_s3 import service_s3
    import importlib.util
    spec = importlib.util.spec_from_file_location("service_s3.py", "C:/Users/Hp/Documents/GitHub/aws-python/awsPy/aws_s3/service_s3.py")
    service_s3 = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(service_s3)

    # Connect to S3
    import os
    from pathlib import Path

    bucket = 'calfdata'
    path = os.getcwd()
    parent_path = str(Path(path).parent)
    path_cred = "{}/programme_matching/credential_AWS.json".format(parent_path)

    con = aws_connector.aws_instantiate(credential = path_cred,
                                            region = 'eu-west-3')
    client= con.client_boto()
    s3 = service_s3.connect_S3(client = client,
                          bucket = 'calfdata')

    
    list_files=s3.list_files_s3(prefix_)

    # Download files from s3
    for file in list_files:
        if file.endswith('.csv'):
            # Download
            s3.download_file(file)

            # Move a file by renaming it's path
            filename = os.path.split(file)[1]
            dest_full_path=os.path.join(download_to_path, filename)
            try:
                os.rename(filename, dest_full_path)
            except:
                os.remove(filename)#fichier déjà existant

    return True

In [ ]:
prefix_='INPI/TC_1/Flux/2018'
download_to_path='flux-old-pour-script'
download_folder_s3(prefix_,download_to_path)